In [7]:
from arepa.load_data.load_data import dataset
from arepa.features.features import structure
from arepa.features.features import gen_dummy
from arepa.features.features import transform_kg
from arepa.pre_processor.pre_processor import preprocessor
from arepa.model.model import model


In [17]:
import warnings
warnings.filterwarnings("ignore")
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [8]:
diabetes = dataset('sample_diabetes_mellitus_data.csv')
df_train = diabetes.train
df_test = diabetes.test

In [11]:
df_train = preprocessor(df_train,['age' , 'gender','ethnicity']).drop_nan()
df_train = preprocessor(df_train,['height' , 'weight']).fill_mean()
df_test = preprocessor(df_test,['age' , 'gender','ethnicity']).drop_nan()
df_test = preprocessor(df_test,['height' , 'weight']).fill_mean()

In [12]:
print('age:', df_train['age'].isnull().sum())
print('gender:', df_train['gender'].isnull().sum())
print('ethnicity:', df_train['ethnicity'].isnull().sum())
print('height:', df_train['height'].isnull().sum())
print('weight:', df_train['weight'].isnull().sum())
print('age:', df_test['age'].isnull().sum())
print('gender:', df_test['gender'].isnull().sum())
print('ethnicity:', df_test['ethnicity'].isnull().sum())
print('height:', df_test['height'].isnull().sum())
print('weight:', df_test['weight'].isnull().sum())

age: 0
gender: 0
ethnicity: 0
height: 0
weight: 0
age: 0
gender: 0
ethnicity: 0
height: 0
weight: 0


In [13]:
## train
df_train = gen_dummy(df_train,['gender','ethnicity']).process()
df_train = transform_kg(df_train,['weight']).process()


## test
df_test = gen_dummy(df_test,['gender','ethnicity']).process()
df_test = transform_kg(df_test,['weight']).process()

In [16]:
f = ['age', 'height', 'weight', 'aids', 'cirrhosis', 'hepatic_failure','immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']
t = ['diabetes_mellitus']
h = {'n_estimators': 200, 'max_depth':2, 'random_state':1}
RFM = model(RandomForestClassifier, df_train, f, t, h, df_test)
y_test_prob = RFM.predict(df_test, f)
print(y_test_prob)

[0.69139015 0.77754172 0.69472355 ... 0.77137552 0.68924726 0.82585423]


In [19]:
df_test['y_test_prob']= y_test_prob

def roc_score(y_test, y_test_prob):
    score_test = roc_auc_score(y_test, y_test_prob)
    return score_test

roc_score(df_test['diabetes_mellitus'],df_test['y_test_prob'])

0.3616730638042558